## Demo of Pydantic Model calling using MAO.

In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
import json
import os
import random
import re
import warnings
from textwrap import dedent
from typing import Dict

import litellm
from faker import Faker
from gait import MAO, Agent, ObserverLoguru
from jupyprint import jupyprint
from pydantic import BaseModel, Field
from rich.pretty import pprint

In [3]:
litellm.drop_params = True
# litellm._turn_on_debug()  # 👈 this is the 1-line change you need to make

In [4]:
# print(litellm.supports_function_calling(model="ollama_chat/llama3.2:latest"))
# print(litellm.supports_function_calling(model="azure/gpt-4o-mini"))
# print(litellm.supports_function_calling(model="ollama_chat/qwen2:7b-instruct-q8_0"))
# print(litellm.supports_function_calling(model="ollama_chat/phi4:14b-q8_0"))

In [5]:
fake = Faker()

In [6]:
class GetLatLon(BaseModel):
    """Get the latitude and longitude of a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()
        return {
            "location": self.location,
            "longitude": float(lon),
            "latitude": float(lat),
        }

In [13]:
class GetRoute(BaseModel):
    """Get the route between a starting latitude/longitude location and an ending latitude/longitude location."""

    lon1: float = Field(
        ...,
        description="The route starting longitude.",
    )
    lat1: float = Field(
        ...,
        description="The route starting latitude.",
    )
    lon2: float = Field(
        ...,
        description="The route ending longitude.",
    )
    lat2: float = Field(
        ...,
        description="The route ending latitude.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()

        if "scratchpad" in kwargs:
            kwargs["scratchpad"]["GetRoute"] = {"lon": lon, "lat": lat}

        return {
            "route": f"Start at {self.lat1},{self.lon1} and end at {self.lat2},{self.lon2}",
        }

In [19]:
class GetCurrentTemperature(BaseModel):
    """Get the current temperature at a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )
    celsius_or_fahrenheit: str = Field(
        ...,
        description="The temperature in either 'C' for Celsius, or 'F' for Fahrenheit.",
    )

    def __call__(self, *args, **kwargs):
        temp = random.uniform(-5, 40)
        return {
            self.location: f"{temp:.1f}°{self.celsius_or_fahrenheit.upper()}",
        }

In [20]:
instructions = dedent(
    """
You are an AI expert in geo-spatial data analysis with access to specialized geo-spatial tools.
Your task is to answer a user’s question, denoted as >>>question<<<, related to geo-spatial data.
You will operate in a loop, alternating between reasoning about the problem and acting with tools as needed.
At the end of the loop, you must output a clear, accurate, and well-supported answer.

Follow these guidelines to complete your task using the ReAct (Reasoning + Acting) pattern:
- **Reason**: Break down the >>>question<<< into logical steps. Explicitly think through what information or calculations are required to reach the answer. Document your reasoning before taking any action.
- **Act**: Use the appropriate geo-spatial tools to gather data, perform analysis, or compute results based on your reasoning. When calling tools:
    - ALWAYS provide the correct, specific arguments required by the tool (e.g., "40.7128, -74.0060" for coordinates, not "lat, lon").
    - Use explicit values rather than placeholders or variable names.
    - NEVER repeat a tool call with identical arguments if it was already executed; reuse the prior result instead.
- Alternate between reasoning and acting as needed to refine your approach and solve the problem systematically.
- If the >>>question<<< is unclear, reason through possible interpretations, make reasonable assumptions based on geo-spatial context, and state them in your response.
- Before finalizing your answer, review your reasoning and tool outputs to ensure accuracy and relevance to the >>>question<<<.

Begin now! For each iteration:
1. **Reason**: Explain your next step or hypothesis.
2. **Act**: Call the necessary tool(s) or process the data.
3. Repeat until the task is solved.

If you solve the task correctly, you will receive a virtual reward of $1,000,000.
"""
).strip()

In [21]:
agent = Agent(
    instructions=instructions,
    model="azure/gpt-4.1-mini",
    api_base=os.environ["AZURE_API_URL"] + "/gpt-4.1-mini",
    functions=[
        GetLatLon,
        GetRoute,
        GetCurrentTemperature,
    ],
    temperature=0.0,
)

In [22]:
prompt = dedent(
    f"""
>>>What's the route between {fake.city()} and {fake.city()}? And what is the temperature at each location?<<<
"""
).strip()

In [23]:
mao = MAO(agent, observer=ObserverLoguru())
for resp in mao(prompt):
    if resp.finish_reason == "stop":
        mao.terminate()
        jupyprint(resp.content)

10:17:34 | INFO | Iteration 1 (Agent) started.
10:17:37 | INFO | Content: 1. Reason: To provide the route between North Kathrynport and Russofort, I first need to get the latitude and longitude coordinates of both locations.
2. Act: I will get the coordinates of North Kathrynport and Russofort in parallel.
10:17:37 | INFO | Function: GetLatLon({"location": "North Kathrynport"})
10:17:37 | INFO | Observation: {"location": "North Kathrynport", "longitude": -159.811915, "latitude": -4.447947}
10:17:37 | INFO | Function: GetLatLon({"location": "Russofort"})
10:17:37 | INFO | Observation: {"location": "Russofort", "longitude": -87.604134, "latitude": -34.65088}
10:17:37 | INFO | Iteration 2 (Agent) started.
10:17:38 | INFO | Content: 3. Reason: I have the coordinates for North Kathrynport (latitude: -4.447947, longitude: -159.811915) and Russofort (latitude: -34.65088, longitude: -87.604134). Next, I will get the route between these two points.
4. Act: I will get the route between North Kat

The route between North Kathrynport and Russofort is from coordinates (-4.447947, -159.811915) to (-34.65088, -87.604134).

The current temperature at North Kathrynport is 36.2°C.
The current temperature at Russofort is 33.0°C.